In [55]:
import pandas as pd
from sqlalchemy import func

from db.db import HotelSession
from db.hotel_models import HotelProperty
from db.advito_models import GeoCountry, GeoState

from fuzzywuzzy import fuzz, process

In [ ]:
hotel_session = HotelSession()
hotel_property = pd.read_sql(
    hotel_session.query(HotelProperty)
    .statement,
    hotel_session.bind)
# print(hotel_property)

In [49]:
df = pd.read_excel(
    'Hotel Agency Detailed SAMPLE - Client C  July-DeC 2019 BCD 1-27-20.xlsx',
    nrows=10,
    sheet_name='Transaction Template',
    dtype=str)
print(df.shape)
print(df)

(10, 50)
                    Chain Name Brand Code          Brand Name  \
0             Hilton Worldwide         HX        Hampton Inns   
1  Marriott International Inc.         MC     Marriott Hotels   
2            Sabre Hospitality         YX   Sabre Hospitality   
3                  TravelClick         WV         Travelclick   
4             Hilton Worldwide         HX        Hampton Inns   
5  Marriott International Inc.         RC      Residence Inns   
6                 Hyatt Hotels         HY        Hyatt Hotels   
7             Hilton Worldwide         GI  Hilton Garden Inns   
8  Marriott International Inc.         RC      Residence Inns   
9  Marriott International Inc.         RC      Residence Inns   

  Internal Hotel Code Sabre Property ID Apollo Property ID  \
0                 NaN           9999998              13686   
1                 NaN           0013110              14014   
2                 NaN           0052896              59694   
3                 NaN      

In [12]:
def __match_ids(row, hotel_property):
    """
    Return (number of matched ids, msg)
    number_of_matched_ids: -1 if error else 0-4 
    msg: error message if error else hotel_property_id
    """
    # create ids dict 
    ids_dict = {
        'id_amadeus': row['Amadeus Property ID'],
        'id_apollo': row['Apollo Property ID'],
        'id_sabre': row['Sabre Property ID'],
        'id_worldspan': row['WorldSpan Property ID']
    }
    # check if ids are present in hotel_property
    tmp = hotel_property[
        (hotel_property['id_amadeus'] == ids_dict['id_amadeus'])
        | (hotel_property['id_apollo'] == ids_dict['id_apollo'])
        | (hotel_property['id_sabre'] == ids_dict['id_sabre'])
        | (hotel_property['id_worldspan'] == ids_dict['id_worldspan'])
    ].reset_index()
    # none of the records are matched
    if len(tmp) == 0:
        ret = (0, None)
    # only one record is matched
    if len(tmp) == 1:
        # get number of matched ids
        ids_matched_no = 0
        for key, value in ids_dict.items():
            if value == tmp[key][0]:
                ids_matched_no += 1
        ret = (ids_matched_no, tmp['id'][0])
    # more than one record is matched
    elif len(tmp) > 1:
        msg = 'Keys present for more than one record in hotel_property: {}'.format(
            ','.join(['{} {}'.format(key, value) for key, value in ids_dict.items()]))
        ret = (-1, msg)
    print(ret)
    return ret

In [66]:
def __fuzzy_match(var, choices, limit=50):
    print('\n\n{}'.format(var))
    ret = list()
    
    ratio = process.extract(var, choices, scorer=fuzz.ratio, limit=limit)
    print('Ratio')
    ret.extend(ratio)
    for item in ratio:
        print(item)
        
    partial_ratio = process.extract(var, choices, scorer=fuzz.partial_ratio, limit=limit)
    ret.extend(partial_ratio)
    print('Partial Ratio')
    for item in partial_ratio:
        print(item)
        
    token_set_ratio = process.extract(var, choices, scorer=fuzz.token_set_ratio, limit=limit)
    ret.extend(token_set_ratio)
    print('Token Set Ratio')
    for item in token_set_ratio:
        print(item)

    return ret
    
    
def _match(row, hotel_property):
    matched = False
    matched_id = None
    ret = None

    ids_matched_no, msg = __match_ids(row, hotel_property)
    print('Number of matched IDs: {}'.format(ids_matched_no))
    
    # check if error happened
    if ids_matched_no == -1:
        ret = msg
    # 1st condition -> 2 or more IDs matched
    elif ids_matched_no >= 2:
        matched = True
        matched_id = msg
    # continue with algorithm for ids_matched in 0 or 1
    else:
        # match hotel_name which is needed for 2nd, 3rd and 4th condition
        # check if there is hotel_property with exact name which will satisfy
        # 2nd condition -> 1 ID and 75% hotel_name
        hp_hn_match_objs = (
            hotel_session.query(HotelProperty)
            .filter(HotelProperty.property_name == row['Hotel Name'])
            .all()
        )
        print('DEBUG')
        print(hp_hn_match_objs)
        return
        if len(hp_hn_match_objs) == 1 and ids_matched == 1:
            print('Hotel Name 100% match found in hotel_property')
            matched = True
            matched_id = hp_match_obj[0].id
    
#         hp_hn_match_objs = hotel_property[hotel_property['property_name'] == row['Hotel Name']]
#         if hp_hn_match_objs.shape[0] == 1 and ids_matched == 1:
#             print('Hotel Name 100% match found in hotel_property')
#             matched = True
#             matched_id = hp_match_obj['id']
        # use Fuzzy to match hotel_name
        else:
            hp_property_names = (
                
            )
            
            hotel_name_matches = __fuzzy_match(row['Hotel Name'], hotel_property['property_name'])
            
#             # 2nd condition -> 1 ID and 75% hotel_name
#             score_treshold = 75
#             # filter hotel name match ids based on score_treshold
#             hn_match_ids = set(
#                 [hp_id for hp_hotel_name, hp_id, score in hotel_name_matches 
#                  if score >= score_treshold])
#             # cross match on hp_match_ids_set and matched_ids
#             tmp = hotel_property[
#                 (hotel_property['id'].isin(hn_match_ids))
#                 | (hotel_property['id_amadeus'] == row['Amadeus Property ID'])
#                 | (hotel_property['id_apollo'] == row['Apollo Property ID'])
#                 | (hotel_property['id_sabre'] == row['Sabre Property ID'])
#                 | (hotel_property['id_worldspan'] == row['WorldSpan Property ID'])
#             ]
#             print(tmp)
#             print(tmp.shape)
#             if tmp.shape[0] == 1:
#                 matched = True
#                 matched_id = tmp.reset_index()['id'][0]
#                 break
            
#             # 3rd condition -> 100% hotel_name, city, state and country code
#             score_treshold = 100
#             # filter hotel name match ids based on score_treshold
#             hn_match_ids = set(
#                 [hp_id for hp_hotel_name, hp_id, score in hotel_name_matches 
#                  if score >= score_treshold])
#             # cross match on hp_match_ids_set and matched_ids
#             query = (
#                 hotel_session.query(HotelProperty)
#                 .filter(HotelProperty.id.in_(hn_match_ids))
#                 .filter(func.lower(HotelProperty.city) == row['City Name'].lower())
#             )
#             if row['Country Code']:
#                 query = (
#                     query
#                     .join(GeoCountry,
#                           GeoCountry.id == HotelProperty.geo_country_id)
#                     .filter(
#                         (GeoCountry.country_code_2char == row['Country Code'])
#                         | (GeoCountry.country_code_3char == row['Country Code'])
#                         | (GeoCountry.country_code_numeric == row['Country Code']))
#                 )
#             if row['State Code']:
#                 query = (
#                     query
#                     .join(GeoState,
#                          GeoState.id == HotelProperty.geo_state_id)
#                     .filter(GeoState.state_code == row['State Code'])
#                 )
#             tmp = query.one()
#             print(tmp)
            
#             tmp = hotel_property[
#                 (hotel_property['id'].isin(hn_match_ids))
#                 | (hotel_property['city'].lower() == row['City Name'].lower())
#                 | (hotel_property['country_code'] == row['Country Code'])
#                 | (hotel_property['id_sabre'] == row['Sabre Property ID'])
#                 | (hotel_property['id_worldspan'] == row['WorldSpan Property ID'])
#             ]
#             print(tmp)
#             print(tmp.shape)
#             if tmp.shape[0] == 1:
#                 matched = True
#                 matched_id = tmp.reset_index()['id'][0]
#                 break
            
            
            
            # 4th condition -> 90% hotel_name, 90% clean_phone
            score_treshold = 90
            if row['Phone Number']:
                # clean phone
                phone = (
                    row['Phone Number']
                    .strip()
                    .replace('/', '')
                    .replace('-', '')
                    .replace(' ', '')
                )
                # filter hotel name match ids based on score_treshold
                hn_match_ids = set(
                    [hp_id for hp_hotel_name, hp_id, score in hotel_name_matches 
                     if score >= score_treshold])
                # cross match on hp_match_ids_set and phone
                tmp = (
                    hotel_session.query(HotelProperty)
                    .filter(
                        (HotelProperty.phone_primary == phone)
                        | (HotelProperty.phone_fax == phone))
                    .all()
                )
                print(tmp)
                print(tmp.shape)
                if len(tmp) == 1:
                    matched = True
                    matched_id = tmp[0].id
#                     break
            
    return matched_id if matched else ret
    
    


df.apply(lambda row: _match(row, hotel_property), axis=1)



(1, 590464)
Number of matched IDs: 1


InternalError: (psycopg2.errors.InFailedSqlTransaction) current transaction is aborted, commands ignored until end of transaction block

[SQL: SELECT hotel_property.id AS hotel_property_id, hotel_property.hotel_chain_id AS hotel_property_hotel_chain_id, hotel_property.property_name AS hotel_property_property_name, hotel_property.property_tag AS hotel_property_property_tag, hotel_property.property_note AS hotel_property_property_note, hotel_property.id_hmf AS hotel_property_id_hmf, hotel_property.id_lanyon AS hotel_property_id_lanyon, hotel_property.id_max AS hotel_property_id_max, hotel_property.id_travelclick AS hotel_property_id_travelclick, hotel_property.id_amadeus AS hotel_property_id_amadeus, hotel_property.id_apollo AS hotel_property_id_apollo, hotel_property.id_sabre AS hotel_property_id_sabre, hotel_property.id_worldspan AS hotel_property_id_worldspan, hotel_property.address1 AS hotel_property_address1, hotel_property.address2 AS hotel_property_address2, hotel_property.city AS

In [17]:
hotel_property.columns.to_list()


['id',
 'hotel_chain_id',
 'property_name',
 'property_tag',
 'property_note',
 'id_hmf',
 'id_lanyon',
 'id_max',
 'id_travelclick',
 'id_amadeus',
 'id_apollo',
 'id_sabre',
 'id_worldspan',
 'address1',
 'address2',
 'city',
 'state',
 'postal_code',
 'geo_city_id',
 'geo_state_id',
 'geo_country_id',
 'regions_id',
 'latitude',
 'longitude',
 'phone_primary',
 'phone_fax',
 'website',
 'email_primary',
 'minority_owner_code',
 'country',
 'olset_rating',
 'id_advito_property_2005',
 'created',
 'modified']